# Motivation

Perhaps a more practice-based approach could be more fruitful. can use textbook & images as RAG dataset.

  1. Initially tried creating LLM for text modality and separate vision encoder / interpreter as vision modality. Necessitated scraping of textbooks as data for LLM backbone and VQA model. still not too sure how to go about encoding images.
  2. Ran into 2 main problems: (a) how do you generate questions (specifically about images) and (b) fine tuning. Did research on question generation models and created a preliminary dataset of questions and answers based on the textbook but still need to explore fine tuning MedFlamingo model specifically.
  3. The textbook isn't super detailed and the scraper isn't perfect - I decided to look into other alternatives for data and QA, starting with a Huggingface dataset with real doctor notes and prognosises. Managed to create this notebook as a pure QA model, which I can use for RAG. Should use textbook data for model fine tuning and this dataset for RAG. **Explore: Evaluating accuracy, how to add image encoding, RAG + utilizing huggingface dataset for QG.**
  4. Need to further explore fine tuning MedFlamingo specifically. To do so, need more valid (image : relevant question : answer) pairings. Should scrape more textbooks. **Resources to explore:**
- fine tuning vid tut: https://www.youtube.com/watch?v=eC6Hd1hFvos&t=1s&ab_channel=ShawhinTalebi
- fine tuning llama model: https://www.datacamp.com/tutorial/fine-tuning-llama-2
- vision model (can look at face VQA as well). keep in mind how you could potentially use this in fine tuning?: https://huggingface.co/docs/transformers/model_doc/vision-encoder-decoder

### TODO:
1. ~~question generation using this notebook's dataset~~ --> does not work; notebook should be used for RAG-esque model similar to this one! (during model fine tuning process?)
2. explore accuracy of this notebook (**using questions generated ?**)
3. fine tuning resources
4. look into rag and find a relevant video
5. vision encoding
6. finalize process + eval more textbooks

# Dependencies

In [51]:
!pip install -U datasets pinecone-client sentence-transformers torch
import numpy as np

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl (209.8 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached triton-2.1.0-0-cp311-cp311-manylin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 13.6 MB/s eta 0:00:000:00:01
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.6
    Uninstalling datasets-2.14.6:
      Successfully uninstalled datasets-2.14.6


# Dataset

In [3]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
radiology_data = load_dataset(
    path = 'Ka4on/radiology',
    split='train',
    streaming=True
).shuffle(seed=960)

/home/developer/anaconda3/envs/andy/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# streaming mode allows us to iterate over the dataset without needing to download it
# show the contents of a single document in the dataset
next(iter(radiology_data))

{'instruction': 'Generate impression based on medical findings.',
 'input': 'Dysarthria. There is no evidence of intracranial hemorrhage, mass, or acute infarct. There are mild scattered foci of cerebral white matter T2 hyperintensity. There is diffuse cerebral volume loss, which is most pronounced in the medial temporal lobes. There is no midline shift or herniation. The major cerebral flow voids are intact. The orbits, skull, paranasal sinuses, and scalp soft tissues are grossly unremarkable.',
 'output': '1. Nonspecific mild scattered foci of cerebral white matter T2 hyperintensity may represent chronic small vessel ischemic disease. Otherwise, no evidence of acute infarction.2. Diffuse cerebral volume loss, which is most pronounced in the medial temporal lobes, which may represent Alzheimer disease in the appropriate clinical setting. '}

In [5]:
from tqdm.auto import tqdm

total_prognosis_count = 50000

counter = 0
docs = []

for d in tqdm(radiology_data, total=total_prognosis_count):
    # extract the fields we need
    doc = {
        "input": d["input"],
        "output": d["output"],
    }
    docs.append(doc)

    # stop iteration once we reach 50k
    if counter == total_prognosis_count:
        break
    counter += 1

100%|██████████████████████████████████| 50000/50000 [00:04<00:00, 10365.67it/s]


In [6]:
import pandas as pd

df = pd.DataFrame(docs)
df.head()

,input,output
0,Dysarthria. There is no evidence of intracrani...,1. Nonspecific mild scattered foci of cerebral...
1,Male 8 years old Reason: Ao root dilatation Le...,1. Status post arterial switch operation.2. No...
2,Pituitary adenoma status post TSH in 11/2013: ...,Interval evolution postoperative findings rela...
3,"History of neuroblastoma of lumbar spine, rela...",1. Postoperative findings related to laminecto...
4,"Encephalopathy: confusion, encephalopathy. Man...",Scattered chronic infarcts and probable chroni...


# Initialize Pinecone idx

In [7]:
import pinecone

# connect to pinecone environment
pinecone.init(
    api_key="05ce3e92-d0da-4ef4-9e3b-2a1ad822689b",
    environment="us-east1-gcp"  # find next to API key in console
)

In [8]:
index_name = "qa"

# check if the abstractive-question-answering index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=768,
        metric="cosine"
    )

# connect to abstractive-question-answering index we created
index = pinecone.Index(index_name)

# Initialize Retriever

##### Retriever Tasks:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. 

Uses SentenceTransformer model based on Microsoft's MPNet as our retriever.

In [9]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# load the retriever model from huggingface model hub
retriever = SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base", device=device)
retriever

/home/developer/anaconda3/envs/andy/lib/python3.11/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


cpu


/home/developer/anaconda3/envs/andy/lib/python3.11/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

# Generate Embeddings and Upsert

In [10]:
# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch["input"].tolist()).tolist()
    # get metadata
    meta = batch.to_dict(orient="records")
    # create unique IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

100%|█████████████████████████████████████████| 782/782 [30:06<00:00,  2.31s/it]


{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 50001}},
 'total_vector_count': 50001}

# Initialize Generator

In [11]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
# TODO: can use LLM trained on textbook, but using RAG with textbooks may be more effective
# going to write down thought process and steps later in documentation
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

In [17]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode([query]).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(xq, top_k=top_k, include_metadata=True)
    return xc

def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m['metadata']['input']}" for m in context]
    # concatinate all context passages
    context = " ".join(context)
    # contcatinate the query and context passages
    query = f"question: {query} context: {context}"
    return query

In [24]:
# example
query = "are there any image findings that are particularly indicative of a positive or negative prognosis for patients with lung cancer?"

context = query_pinecone(query, top_k=1)
print(context)

query = format_query(query, context['matches'])
print(query)

{'matches': [{'id': '39306',
              'metadata': {'input': 'Evaluate disease status for clinical '
                                    'trial. Provide 3-D measurements are all '
                                    'reference lesions and compare prior CT. '
                                    'Lung cancer status post 22 cycles of '
                                    'chemotherapy. CHEST:LUNGS AND PLEURA: '
                                    'Right upper lobe mass measures 2.3 x 1.8 '
                                    'cm (image 30; series 4), not '
                                    'significantly changed from previous '
                                    'study.MEDIASTINUM AND HILA: Small '
                                    'pericardial effusion. Stable calcified '
                                    'right hilar and subcarinal lymph '
                                    'nodes.CHEST WALL: No significant '
                                    'abnormality notedABDOMEN:LIVE

In [ ]:
# TODO: would insert RAG functionality and more fine tuning here - explore how!

# Testing / Eval

In [40]:
def generate_answer(query, evaluation=False):
    """
    Given a query, returns relevant answer as a string. 
    If evaluation is true, also returns vector match score
    """
    # establish context from pinecone query
    context = query_pinecone(query, top_k=5)
    query = format_query(query, context["matches"])
    score = context["matches"][0]["score"]
    
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    
    # return type depends on evaluation status
    if evaluation:
        return (answer, score)
    else:
        return answer

In [41]:
query = "are there any image findings that are particularly indicative of a positive or negative prognosis for patients with lung cancer?"

answer = generate_answer(query)

print(answer)

There are a couple of things to consider. The first is that there is a lot of variability in the image quality. The second is that there is a lot of variability in the image quality


# Evaluation 

i. how similar a variety of queries are on average to vectors in our pinecone index

In [46]:
# import dataset of questions generated
qa_dataset = pd.read_csv('qa_dataset.csv')
qa_dataset = qa_dataset.drop('Unnamed: 0', axis='columns')
qa_dataset.head()

,question,answer
0,<pad> question: What is the name of the angiog...,[STARTUP]
1,<pad> question: What is a common complication ...,Air embolism
2,<pad> question: What is SVC?</s>,superior vena cava (s Vc)
3,<pad> question: What is the role of interventi...,Hepatic artery aneurysmMesenteric ischemiaAcut...
4,<pad> question: What is the name of the diseas...,Atherosclerotic renal artery stenosisFibromusc...


In [ ]:
# generating responses using model

generative_responses, scores = [], []

for q in tqdm(qa_dataset['question']):
    generative_response, score = generate_answer(q, evaluation=True)
    generative_responses.append(generative_response)
    scores.append(score)

qa_dataset['generative response'] = generative_responses
qa_dataset['vector similarity score'] = scores

qa_dataset.head()

In [58]:
# exporting dataset for future use
qa_dataset.to_csv('qa_dataset_with_qg.csv')

In [52]:
# evaluating vector context similarity

print(f"average vector similarity score: {np.mean(qa_dataset['vector similarity score'])}")

average vector similarity score: 0.592321467470339


,question,answer,generative response,vector similarity score
0,<pad> question: What is the name of the angiog...,[STARTUP],The angiogram shows the four phases of vascula...,0.567982
1,<pad> question: What is a common complication ...,Air embolism,The most common complication of embolization i...,0.640801
2,<pad> question: What is SVC?</s>,superior vena cava (s Vc),SVC is a condition where the blood vessels in ...,0.502530
3,<pad> question: What is the role of interventi...,Hepatic artery aneurysmMesenteric ischemiaAcut...,Interventional radiology is used in a variety ...,0.594041
4,<pad> question: What is the name of the diseas...,Atherosclerotic renal artery stenosisFibromusc...,The name of the disease that causes the appear...,0.627335


ii. accuracy of responses 

In [70]:
# manual sampling / examination of questions with their responses

def examine(i):
    assert i in range(len(qa_dataset))
    print('Q:', qa_dataset['question'][i])
    print('A:', qa_dataset['generative response'][i], '\n')

np.random.seed(42)
for i in np.random.randint(0, len(qa_dataset), 5):
    examine(i)

Q: <pad> question: What is a nonmucinous BAC?</s>
A: A nonmucinous BAC is a BAC that is not caused by a blood clot. It is a BAC that is not caused by a blood clot. 

Q: <pad> question: What is the name of the disease that causes multiple intraductal papillomas?</s>
A: It's a type of breast cancer. It's a type of breast cancer that occurs in the breast tissue. It's a type of breast cancer that occurs in the breast tissue. It's 

Q: <pad> question: What is the name of the condition that can cause medullary calcification?</s>
A: Medullary calcification is a condition that can occur in a number of different ways. It can be caused by a number of things, including: 1. Cardiac blockage 2. 

Q: <pad> question: What two areas of the brain are referred to as CT perfusion?</s>
A: I'm not sure what you mean by perfusion, but I'll try to answer your question. The brain is made up of a bunch of cells called neurons. Each neuron is responsible for 

Q: <pad> question: What is the definition of?</s>
A

In [43]:
# TODO: explore RLHF or outside trained reward model; 
# can also eval based off answers of our qa dataset, but I'm not sure how accurate those are

# Appendix

### source / references
- https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/search/question-answering/abstractive-question-answering.ipynb#scrollTo=6iFLxPPvx2Tm